In [37]:
import glob
from astropy.table import Table
import fitsio
import os
import matplotlib.pyplot as plt
from datetime import datetime
import pytz
import numpy as np
from scipy import stats
import pandas as pd

In [46]:
def process_fits_file(fafn):
    tile_data = {}
    _, h = fitsio.read(fafn, "PRIMARY", header=True)
    
    try:
        program = h["FAPRGRM"]
    except KeyError:
        print('failed FAPRGM with', fafn)
        return None
        
    try:
        date_string = h["NOWTIME"]
    except KeyError:
        print('failed NOW TIME with', fafn)
        return None
    
    tile_data["program"] = h["FAPRGRM"]
    tile_data["ra"] = h["TILERA"]
    tile_data["dec"] = h["TILEDEC"]
    
    datetime_obj = datetime.strptime(date_string, '%Y-%m-%dT%H:%M:%S%z').replace(tzinfo=pytz.UTC)
    print(datetime_obj)
    reference_date = datetime(2021, 5, 14, tzinfo=pytz.UTC) # The start of the main survey
    days_difference = (datetime_obj - reference_date).total_seconds()/(24.0*3600.0)
    tile_data["days_difference"] = days_difference
    
    d = Table(fitsio.read(fafn, "FIBERASSIGN"))

    sky = d["OBJTYPE"] == "SKY"
    good_fibers = d["FIBERSTATUS"]==0 
    expid_str = fafn.split(os.path.sep)[-2]
    
    tile_data["nsky"] = np.count_nonzero(sky)
    tile_data["nskygoodfiber"] = np.count_nonzero(sky&good_fibers)
    tile_data["nskybadfiber"] = np.count_nonzero(sky&(~good_fibers))
    tile_data["expid"] = int(expid_str)
    tile_data["nlowp"] = np.count_nonzero(d["PRIORITY"]<1400)
    tile_data["nverylowp"] = np.count_nonzero(d["PRIORITY"]<100)
    
    return tile_data

def process_fits_files(fafns):
    tile_nsky = []
    tile_nsky_goodfiber = []
    tile_nsky_badfiber = []
    tile_nlowp = []
    tile_nverylowp = []
    expid = []
    tile_program = []
    tile_ra = []
    tile_dec = []
    tile_days = []
    

    for fafn in fafns:
        tile_data = process_fits_file(fafn)
        if tile_data:
            tile_program.append(tile_data["program"])
            tile_ra.append(tile_data["ra"])
            tile_dec.append(tile_data["dec"])
            tile_days.append(tile_data["days_difference"])
            tile_nsky.append(tile_data["nsky"])
            tile_nsky_goodfiber.append(tile_data["nskygoodfiber"])
            tile_nsky_badfiber.append(tile_data["nskybadfiber"])
            tile_nlowp.append(tile_data["nlowp"])
            tile_nverylowp.append(tile_data["nverylowp"])
            expid.append(tile_data["expid"])
            
    data = {"tile_nsky": np.array(tile_nsky),
            "tile_nsky_goodfiber":np.array(tile_nsky_goodfiber),
            "tile_nsky_badfiber":np.array(tile_nsky_badfiber),
            "tile_nlowp":np.array(tile_nlowp),
            "tile_nverylowp":np.array(tile_nverylowp),
            "expid": np.array(expid),
            "tile_program": np.array(tile_program),
            "tile_ra": np.array(tile_ra),
            "tile_dec": np.array(tile_dec),
            "tile_days": np.array(tile_days)
    }
    df = pd.DataFrame(data)
    return df

def process_year(year):
    fafns = sorted(glob.glob("/global/cfs/cdirs/desi/spectro/data/{}*/????????/fiberassign*fits.gz".format(year)))
    print('gathered {} files'.format(len(fafns)))
    df_year = process_fits_files(fafns)
    filename = '{}_nsky_lowp_data.csv'.format(year)
    df_year.to_csv(filename, index=False)

In [ ]:
for year in [2021, 2022, 2023, 2024]:
    process_year(year)

gathered 7292 files
failed FAPRGM with /global/cfs/cdirs/desi/spectro/data/20210106/00071093/fiberassign-080641.fits.gz
failed FAPRGM with /global/cfs/cdirs/desi/spectro/data/20210106/00071094/fiberassign-080641.fits.gz
failed FAPRGM with /global/cfs/cdirs/desi/spectro/data/20210106/00071095/fiberassign-080641.fits.gz
failed FAPRGM with /global/cfs/cdirs/desi/spectro/data/20210106/00071096/fiberassign-080641.fits.gz
failed FAPRGM with /global/cfs/cdirs/desi/spectro/data/20210106/00071097/fiberassign-080641.fits.gz
failed FAPRGM with /global/cfs/cdirs/desi/spectro/data/20210106/00071099/fiberassign-080641.fits.gz
failed FAPRGM with /global/cfs/cdirs/desi/spectro/data/20210106/00071100/fiberassign-080641.fits.gz
failed FAPRGM with /global/cfs/cdirs/desi/spectro/data/20210106/00071101/fiberassign-080642.fits.gz
failed FAPRGM with /global/cfs/cdirs/desi/spectro/data/20210106/00071102/fiberassign-080642.fits.gz
failed FAPRGM with /global/cfs/cdirs/desi/spectro/data/20210106/00071103/fiberas